In [12]:
import arrow
import numpy as np
import numpy.random as rd
import time
import json
import random

In [2]:
names = ["Marquita Fleur",
"Alexandria Witzke",
"Hayden Hurlburt",
"Nicol Thoman",
"Shasta Handy",
"Jacalyn Zhou",
"Ji Lanman",
"Tari Biller",
"Selene Windley",
"Nyla Couey",
"Rolf Chavarria",
"Evelina Hellwig",
"Jenni Heilig",
"Branda Siciliano",
"Charles Hogue",
"Trang Macha",
"Jenell Chadburn",
"Janessa Clem",
"Toccara Carlen",
"Malorie Brogden",
"Sana Brinson",
"Nolan Ernest",
"Ellsworth Keeble",
"Carey Lall",
"Audra Stephenson",
"Mardell Edlin",
"Rubye Lohr",
"Marylynn Eberhart",
"Dionna Greenwald",
"Mickey Yen",
"Shella Kelsch",
"Roxann Xiong",
"Kenia Tierney",
"Henriette Jefferson",
"Tania Adcock",
"Fidelia Mckennon",
"Leon Kehoe",
"Obdulia Smothers",
"Sixta Prado",
"Lyn Abernethy",
"Alane Michaux",
"Gema Reisch",
"Tama Pompei",
"Herta Bennet",
"Lilliana Mcwilliams",
"Shayla Magwood",
"Vikki Blackerby",
"Teresita Willsey",
"Gertha Bellew",
"Nick Como"]

In [3]:
one_liners = [
    "54 W hx colon cancer admitted from PCP with uptrending transaminitis, MRI confirms HCC",
    "21M POD#0 s/p uncomplicated lap appy, tolerating sips with no n/v - possible discharge today",
    "32M hx polysubstance abuse p/w stab wound x 4 to abdomen now POD#7 s/p ex/lap, repair of multiple enterotomies, small bowel resection / primary re-anastomosis now tolerating regular diet",
    "6M with multiple psychiatric comorbitidies p/w self-inflicted stab wound to chest, now POD#2 from VATS decortication / evacuation of hematoma and placement of chest tubes x 2 undergoing intrapleural TPA protocol",
    "43W POD#1 s/p lap ccy for acute cholecystitis awaiting discharge",
    "53M unrestrained passenger MVC who sustained multiple traumatic injuries now s/p L femur IM nail by ortho awaiting acute rehab placement",
]

In [4]:
medications = ['Diovan','Levemir','Gleevec','Herceptin','Vyvanse','Lucentis','Zetia','Combivent','Symbicort','Namenda','NovoLog','Xarelto','NovoLog','Humalog','Suboxone','Viagra','Seroquel','ONE','AndroGel','Enoxaparin','Methylphenidate','ProAir','Alimta','Victoza','Synagis','Avonex','Renvela','Rebif','Cialis','Gilenya','Nasonex','Stelara','Restasis','Budesonide','Acetaminophen','Flovent','Lovaza','Prezista','Isentress','Janumet','Procrit','Doxycycline','Orencia','Amphetamine','Vesicare','Dexilant','Humalog','Neupogen','Lidocaine','Lunesta','Fenofibrate','Zytiga','Reyataz','Sensipar','Metoprolol','Invega','Synthroid','Avonex','Prevnar','Xolair','Lipitor','levothyroxine','Benicar','Stribild','Zostavax','Pradaxa','Vytorin','Tamiflu','Xgeva','Evista','Xeloda','Aranesp', 'Ventolin','divalproex','Afinitor','Betaseron','Adderall','Complera']

In [5]:
"""for our purposes make max length of stay 7 days and 3 minimum"""
los_max = 3
los_min = 7

class Lab(object):
    """ Simulate a lab draw """
    def __init__(self, lo, hi, sd):
        self.lo = lo
        self.hi = hi
        self.sd = sd
        
    def draw_lab(self):
        mean = self.lo + (self.hi - self.lo)
        return np.random.normal(mean, self.sd)

In [6]:
na_lo = 135
na_hi = 145
k_lo = 3.5
k_hi = 5
cl_lo = 96
cl_hi = 106
bic_lo = 22
bic_hi = 29
bun_lo = 7
bun_hi = 20
cr_lo = 0.6
cr_hi = 1.2
gluc_lo = 70
gluc_hi = 100

In [7]:
wbc_lo = 4
wbc_hi = 11
hgb_lo = 6
hgb_hi = 15
hct_lo = 18
hct_hi = 45
plt_lo = 150
plt_hi = 320

In [8]:
lab_sbp = Lab(100, 140, (-1) * (100-140)/4)
lab_dbp = Lab(80,100, (-1) * (80-100)/4)
lab_hr = Lab(50, 100, (-1) * (50-100)/4)
lab_sat = Lab(90, 100, (-1) * (90- 100)/4)

In [9]:
lab_na = Lab(na_lo, na_hi, (-1) * (na_lo - na_hi)/4)
lab_k = Lab(k_lo, k_hi, (-1) * (k_lo - k_hi)/4)
lab_cl = Lab(cl_lo, cl_hi, (-1) * (cl_lo - cl_hi)/4)
lab_bic = Lab(bic_lo, bic_hi, (-1) * (bic_lo - bic_hi)/4)
lab_bun = Lab(bun_lo, bun_hi, (-1) * (bun_lo - bun_hi)/4)
lab_cr = Lab(cr_lo, cr_hi, (-1) * (cr_lo - cr_hi)/4)
lab_gluc = Lab(gluc_lo, gluc_hi, (-1) * (gluc_lo - gluc_hi)/4)

def sd_from_lo_hi(l,h):
    return ((-1) * (l - h)/4)
    
lab_wbc = Lab(wbc_lo, wbc_hi, sd_from_lo_hi(wbc_lo,wbc_hi))
lab_hgb = Lab(hgb_lo, hgb_hi, sd_from_lo_hi(hgb_lo,hgb_hi))
lab_hct = Lab(hct_lo, hct_hi, sd_from_lo_hi(hct_lo,hct_hi))
lab_plt = Lab(plt_lo, plt_hi, sd_from_lo_hi(plt_lo,plt_hi))

In [10]:
class Person(object):
    """ Simulate a person """
    def __init__(self, mrn):
        self.data = {}
        self.los = 0
        self.admit_time = 0
        self.now = arrow.utcnow()
        self.mrn = mrn
        #self.admit_time = {}
       
        self.run_person()
        
    def pick_name(self):
        self.data['name'] = rd.choice(names)
        self.data['mrn'] = self.mrn

    def generate_med_item(self):
        #n = rd.randint(5, 15)
        med_item = {}
        med_item['name'] = rd.choice(medications)
        med_item['context'] = rd.choice(['home', 'hospital'])
        #[{rd.choice(medications): {'context' : rd.choice(['home', 'hospital'])}}  for i in range (0,n)]
        #meds_context = [rd.choice(['home', 'hospital']) for i in range (0,n)]
        #meds_data = []
        #for i in range (0,n):
        #    meds_data += [{}]
        #self.data['meds']
        #print(meds_list)
        #return meds_list
        return med_item
    

    def pick_one_liner(self):
        self.data['one_liner'] = rd.choice(one_liners)        
        
    def generate_los(self):
        self.los = rd.randint(los_max, los_min)
        self.admit_time = self.now.replace(days=-(self.los))
        #self.data['los'] = rd.randint(los_max, los_min)

    def generate_bmp_instance(self, timestamp):
        bmp = {}
        bmp['event'] = 'bmp'
        bmp['timestamp'] = timestamp
        bmp['na'] = lab_na.draw_lab()
        bmp['k'] = lab_k.draw_lab()
        bmp['cl'] = lab_cl.draw_lab()
        bmp['bic'] = lab_bic.draw_lab()
        bmp['bun'] = lab_bun.draw_lab()
        bmp['cr'] = lab_cr.draw_lab()
        bmp['gluc'] = lab_gluc.draw_lab()
        return bmp
    
    def generate_cbc_instance(self, timestamp):
        cbc = {}
        cbc['event'] = 'cbc'
        cbc['timestamp'] = timestamp
        cbc['wbc'] = lab_wbc.draw_lab()
        cbc['hgb'] = lab_hgb.draw_lab()
        cbc['hct'] = lab_hct.draw_lab()
        cbc['plt'] = lab_plt.draw_lab()
        return cbc

    def generate_vit_instance(self, timestamp):
        vit = {}
        vit['event'] = 'vit'
        vit['timestamp'] = timestamp
        vit['sbp'] = lab_sbp.draw_lab()
        vit['dbp'] = lab_dbp.draw_lab()
        vit['hr'] = lab_hr.draw_lab()
        vit['sat'] = lab_sat.draw_lab()
        return vit
        
    def generate_bmps(self):
        """ generate bmp data """
        utc = self.now
        bmp_data = []
        for i in range(0,self.los):
            past = self.now.replace(days=-(i))
            past_str = past.format('X')
            bmp_data += [self.generate_bmp_instance(past_str)]
        return bmp_data

    def generate_cbcs(self):
        """ generate cbc data """    
        draw_cbc_prop_los = np.random.rand(self.los)
        draw_cbc_ts = [self.admit_time.replace(days=(i*self.los)) for i in draw_cbc_prop_los]
        cbc_data = []
        for i in draw_cbc_ts:
            past_str = i.format('X')
            cbc_data += [self.generate_cbc_instance(past_str)]
        return cbc_data
        
    def generate_vits(self):
        """ generate cbc data"""    
        #assume vitals are drawn 4 times per day, hence self.los * 4
        draw_vit_prop_los = np.random.rand(self.los * 4)
        draw_vit_ts = [self.admit_time.replace(days=(i*self.los)) for i in draw_vit_prop_los]
        vit_data = []
        for i in draw_vit_ts:
            past_str = i.format('X')
            vit_data += [self.generate_vit_instance(past_str)]
        return vit_data

    def generate_meds(self):
        """ generate meds data"""  
        #generate random number of meds in patient's meds list:
        n = rd.randint(5, 15)
        meds_data = []
        for i in range(0,n):
            meds_data += [self.generate_med_item()]
        return meds_data
    
    def collect_labs(self):
        """collect lab data in dict"""
        self.data['labs'] = {}
        self.data['labs']['bmp'] = self.generate_bmps()
        self.data['labs']['cbc'] = self.generate_cbcs()
    
    def collect_vits(self):
        """collect vitals data in dict"""
        self.data['vits'] = {}
        self.data['vits'] = self.generate_vits()    

    def collect_meds(self):
        """collect vitals data in dict"""
        self.data['meds'] = {}
        self.data['meds'] = self.generate_meds()        
    
    def run_person(self):
        self.pick_name()
        self.pick_one_liner()
        self.generate_los()
        
        self.collect_labs()
        self.collect_vits()
        self.collect_meds()
        return
        
        


In [13]:
num_pts = 6
mrns = random.sample(range(9), 6)
data = []
for i in range(0,num_pts):
    p = Person(mrns[i])
    data += [p.data]
    

In [14]:
data

[{'labs': {'bmp': [{'bic': 31.010233968858856,
     'bun': 24.298080664608136,
     'cl': 110.2798121189743,
     'cr': 1.0896181393726276,
     'event': 'bmp',
     'gluc': 100.29712927105011,
     'k': 5.049772817682521,
     'na': 141.8556841724145,
     'timestamp': '1490664648'},
    {'bic': 26.277869875137775,
     'bun': 18.575236345647888,
     'cl': 105.35300556921918,
     'cr': 1.3576145813534346,
     'event': 'bmp',
     'gluc': 105.52478340421519,
     'k': 5.075481492925226,
     'na': 141.96068999924927,
     'timestamp': '1490578248'},
    {'bic': 31.261360316215075,
     'bun': 20.602826989973472,
     'cl': 105.7408113827692,
     'cr': 1.1067359641665142,
     'event': 'bmp',
     'gluc': 95.93270162889345,
     'k': 4.545488626351712,
     'na': 142.65667668741017,
     'timestamp': '1490491848'},
    {'bic': 27.713846307108724,
     'bun': 24.243375147748626,
     'cl': 105.88838409490785,
     'cr': 1.3991077000181784,
     'event': 'bmp',
     'gluc': 99.1705387

In [15]:
with open('Data_3.json', 'w') as outfile:
    json.dump(data, outfile)